In [ ]:
# Strukture Podataka u Seaborn-u

## Seminarski rad - SISJ

### Deo 3: Strukture podataka koje Seaborn može da prihvati

---

## 1. Uvod u strukture podataka

Seaborn je optimizovan za rad sa **pandas DataFrame** objektima, ali može da radi i sa drugim strukturama podataka. Razumevanje različitih formata podataka je ključno za efikasno korišćenje biblioteke.

### 1.1 Podržane strukture podataka:

1. **Pandas DataFrame** - primarni format (preporučeno)
2. **Pandas Series** - za jednostavne 1D podatke
3. **NumPy arrays** - za numeričke podatke
4. **Python lists** - za osnovne podatke
5. **Dictionaries** - za structured podatke

### 1.2 Tidy data princip:

Seaborn preferira **"tidy" (uređene) podatke** gde:
- Svaki red predstavlja jednu **observaciju**
- Svaka kolona predstavlja jednu **varijablu**
- Svaka ćelija sadrži jednu **vrednost**

Ovaj format omogućava najintuitivnije i najfleksibilnije kreiranje grafika.


In [ ]:
# Uvoz potrebnih biblioteka
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# Podešavanje
sns.set_style("whitegrid")
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 8)

# Demonstracija različitih struktura podataka
print("=== DEMONSTRACIJA RAZLIČITIH STRUKTURA PODATAKA ===")

# 1. Pandas DataFrame (preporučeno)
print("\n1. PANDAS DATAFRAME:")
df = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2, 4, 6, 8, 10],
    'group': ['A', 'A', 'B', 'B', 'B']
})
print(df)

# 2. Pandas Series
print("\n2. PANDAS SERIES:")
series = pd.Series([1, 2, 3, 4, 5], name='values')
print(series)

# 3. NumPy arrays
print("\n3. NUMPY ARRAYS:")
array_x = np.array([1, 2, 3, 4, 5])
array_y = np.array([2, 4, 6, 8, 10])
print(f"Array X: {array_x}")
print(f"Array Y: {array_y}")

# 4. Python lists
print("\n4. PYTHON LISTS:")
list_x = [1, 2, 3, 4, 5]
list_y = [2, 4, 6, 8, 10]
print(f"List X: {list_x}")
print(f"List Y: {list_y}")

# 5. Dictionary
print("\n5. DICTIONARY:")
data_dict = {
    'x': [1, 2, 3, 4, 5],
    'y': [2, 4, 6, 8, 10],
    'group': ['A', 'A', 'B', 'B', 'B']
}
print(data_dict)


In [ ]:
# Demonstracija plotova sa različitim strukturama podataka
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Seaborn sa različitim strukturama podataka', fontsize=16)

# 1. DataFrame
sns.scatterplot(data=df, x='x', y='y', hue='group', ax=axes[0,0])
axes[0,0].set_title('DataFrame sa hue grupisanjem')

# 2. Series
sns.histplot(series, ax=axes[0,1])
axes[0,1].set_title('Series histogram')

# 3. NumPy arrays
sns.scatterplot(x=array_x, y=array_y, ax=axes[0,2])
axes[0,2].set_title('NumPy arrays')

# 4. Python lists
sns.lineplot(x=list_x, y=list_y, ax=axes[1,0])
axes[1,0].set_title('Python lists')

# 5. Dictionary direct
sns.boxplot(data=data_dict, y='y', x='group', ax=axes[1,1])
axes[1,1].set_title('Dictionary data')

# 6. Mixed approach - DataFrame converted from dictionary
df_from_dict = pd.DataFrame(data_dict)
sns.violinplot(data=df_from_dict, y='y', x='group', ax=axes[1,2])
axes[1,2].set_title('Dictionary → DataFrame')

plt.tight_layout()
plt.show()

print("ZAKLJUČAK:")
print("Sve strukture podataka rade, ali DataFrame pruža:")
print("- Najbolju fleksibilnost")
print("- Najjasniju sintaksu")
print("- Podršku za kategorijske varijable")
print("- Jednostavno grupisanje i filtriranje")
